### KEEP UI

In [2]:
import pandas as pd
import numpy as np
import streamlit as st
import time
import json
import os
import pathlib
import time
import pyetherbalance
import pandas as pd
import ssl

st.title('Keep AletrBalance ETH (ECDSA)')
eth = st.text_input("Enter you ETH wallet")
df = pd.read_csv('df.csv')
df = df.drop_duplicates(subset=['wallet'])

def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

# def remote_css(url):
#     st.markdown(f'<link href="{url}" rel="stylesheet">', unsafe_allow_html=True)
#
# def icon(icon_name):
#     st.markdown(f'<i class="material-icons">{icon_name}</i>', unsafe_allow_html=True)

local_css("color2.css")


if eth:
    infura_url = 'https://ropsten.infura.io/v3/d6d4edf5380445288c66be54f67d20f1'
    ethereum_address = eth
    if ethereum_address in list(df['wallet']):
        st.write(df[df['wallet']==ethereum_address])
        st.write('Already exist')
        if st.checkbox('Change Treshold'):
            new_t = st.text_input("Enter you new Treshold")
            for index, row in df.iterrows():
                st.write(row['wallet'], row['treshhold'])
                if ethereum_address == row['wallet']:
                    eth_new = index
                    df.loc[eth_new, 'treshhold'] = new_t
                    st.write(df)
                    df.to_csv('df.csv', index=False)
        if st.checkbox('Change mail '):
            new_m = st.text_input("Enter you new mail")
            for index, row in df.iterrows():
                st.write(row['wallet'], row['mail'])
                if ethereum_address == row['wallet']:
                    eth_new = index
                    df.loc[eth_new, 'mail'] = new_m
                    st.write(df)
                    df.to_csv('df.csv', index=False)
    else:
    # Create an pyetherbalance object , pass the infura_url
        ethbalance = pyetherbalance.PyEtherBalance(infura_url)
        # get ether balance
        balance_eth = ethbalance.get_eth_balance(ethereum_address)
        if balance_eth['status'] == 'error':
            st.write('Please enter correct format')
        else:
            # get token balance
            #balance_omg = ethbalance.get_token_balance('OMG', ethereum_address)
            #print(balance_omg)
            Treshold = st.text_input("Enter you Treshold")
            #st.write(Treshold)
            mail = st.text_input("Enter your mail")
            #st.write(mail)
            if mail and Treshold:
            #df = pd.DataFrame(columns=['wallet', 'treshhold','send','mail'])
                df.loc[len(df)] = [eth, Treshold,'0',mail]
                #st.write(df)
                st.balloons()
                st.success('Thank you!')

            df.to_csv('df.csv',index=False)


### KEEP ALARM SCRIPT

In [2]:
import pathlib
import time
import pyetherbalance
import pandas as pd
import ssl
import smtplib

def alarm(b, w, t, s, m , index):
    smtp_server = "smtp.gmail.com"
    port = 587  # For starttls
    sender_email = "KeepNetBazilik@gmail.com"
    password = "********"
    receiver_email = m
    message = 'Alert, your ETH balance for KEEP< %s' % t

    # Create a secure SSL context
    context = ssl.create_default_context()

    # print(b,w,t,s)

    if (s == 1) and (b < t):
        print(w, 'SENDED')
        # continue
    if (s != 1) and (b > t):
        print(w, 'OK')

    if (s != 1) and (b <= t):

        try:
            server = smtplib.SMTP(smtp_server, port)
            server.ehlo()  # Can be omitted
            server.starttls(context=context)  # Secure the connection
            server.ehlo()  # Can be omitted
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message)
            # TODO: Send email here
        except Exception as e:

            # Print any error messages to stdout
            print(e)
        finally:
            server.quit()

        df['send'][index] = 1
        df.to_csv('df.csv',index=False)

        print(w, 'OK')

    if (s == 1) and (b > t):
        df['send'][index] = 0
        df.to_csv('df.csv', index=False)


import time

while True:
    df = pd.read_csv('df.csv')
    df = df.drop_duplicates(subset=['wallet'])
    for index, row in df.iterrows():
        #print(row['wallet'],row['treshhold'])
        infura_url = 'https://ropsten.infura.io/v3/d6d4edf5380445288c66be54f67d20f1'
        ethereum_address = row['wallet']
        #row['wallet'] += 1
        # Create an pyetherbalance object , pass the infura_url
        ethbalance = pyetherbalance.PyEtherBalance(infura_url)
        # get ether balance
        balance_eth = ethbalance.get_eth_balance(ethereum_address)
        if balance_eth['status'] == 'error':
            continue
        #print(balance_eth['balance'],row['wallet'])
        b=balance_eth['balance']
        w = row['wallet']
        t = row['treshhold']
        s= row['send']
        m = row['mail']
        alarm(b,w,t,s,m,index)

        time.sleep(1)
        print(index)
